In [1]:
#imports
import numpy as np
import pandas as pd
#import quandl 
import matplotlib.pyplot as plt
%matplotlib inline
import gspread as gs
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from gspread_formatting.dataframe import format_with_dataframe
from scipy.optimize import minimize
import jupytab
import yfinance as yf

In [1]:
class PortfolioBalancer():
    def __init__(self, df):
        self.df = df
        w_random = np.random.random(18)
        self.weights = np.array(w_random)
    def classifyStocks(self):
        if df['score'].iloc[0] <= 12:
            tickers = ['aapl','adbe','amd','csco', 'fisv', 'ibm', 'intc', 'lrcx', 'msft', 'mu', 'orcl', 'qcom', 'txn', 'nvda', 'fis', 'crm', 'avgo', 'now']
    
        elif (df['score'].iloc[0] > 12) & (df['score'].iloc[0] <= 20):
            tickers = ['csgp','mdu','amd','csco', 'lly', 'ibm', 'intc', 'wba', 'isrg', 'pep', 'orcl', 'qcom', 'aciw', 'nvda', 'qlys', 'crm', 'cost', 'now']

        elif (df['score'].iloc[0] > 20) & (df['score'].iloc[0] <= 28):
            tickers = ['aapl','adbe','amd','csco', 'fisv', 'ibm', 'intc', 'lrcx', 'msft', 'mu', 'orcl', 'nbix', 'txn', 'nvda', 'fis', 'crm', 'avgo', 'saic']

        elif (df['score'].iloc[0] > 29) & (df['score'].iloc[0] <= 34):
            tickers = ['dci','adbe','mrcy','wtrg', 'fisv', 'ibm', 'axs', 'lrcx', 'msft', 'bros', 'orcl', 'pypl', 'txn', 'amrc', 'fis', 'crm', 'chgg', 'now']
            
        elif (df['score'].iloc[0] > 35):
            tickers = ['meta','adbe','amd','csco', 'fisv', 'ibm', 'ale', 'lrcx', 'jazz', 'mu', 'orcl', 'qcom', 'txn', 'nvda', 'fis', 'crm', 'eqix', 'vrsk']

        stock_data = yf.download(tickers,start='2017-01-1', end='2018-12-1')
        stock_data = stock_data.dropna()
        stocks = stock_data['Adj Close']
        return stocks
    
    def calculateSharpeRatio(self):
        # Set seed (optional)
        #np.random.seed(100)

        log_ret = np.log(self.classifyStocks()/self.classifyStocks().shift(1))
        log_ret.head()

        
        # print(weights/np.sum(weights))
        # print('\n\n')

        weight_log = log_ret.mean() * self.weights
        expected_returns = np.sum(weight_log) *252
        # print(expected_returns)
        # print('\n')

        transformed_weight = self.weights.T
        day_scale = 252
        expected_volatility = np.sqrt(np.dot(transformed_weight, np.dot(log_ret.cov() * day_scale, self.weights)))
        # print(expected_volatility)
        # print('\n')

        sharpe_ratio = expected_returns/expected_volatility
        # print(sharpe_ratio)
        num_of_stocks = len(self.classifyStocks().columns)
        all_weights = np.zeros((15000, num_of_stocks))
        ret_arr = np.zeros(15000)
        vol_arr = np.zeros(15000)
        sharpe_arr = np.zeros(15000)
        day_len = 252
        w_transform = self.weights.T
        retention_mean = log_ret.mean()

        for indicator in range(15000):
            # weights = np.array(np.random.random(18))
            all_weights[indicator,:] = self.weights / np.sum(self.weights)
            ret_arr[indicator] = np.sum((retention_mean * self.weights) * day_len)
            vol_arr[indicator] = np.sqrt(np.dot(w_transform, np.dot(log_ret.cov() * day_len, self.weights)))
            sharpe_arr[indicator] = ret_arr[indicator]/vol_arr[indicator]
        return log_ret
            
    def get_rvs(self):
        # weights = np.array(self.calculateSharpeRatio())
        ret = np.sum(self.calculateSharpeRatio().mean() * self.weights) * 252
        vol = np.sqrt(np.dot(self.weights.T, np.dot(self.calculateSharpeRatio().cov() * 252, self.weights)))
        sr = ret/vol
        return np.array([ret,vol,sr])

    def negative_sharpe(self):
        rvs_return = self.get_rvs()[2]
        neg_rvs = rvs_return * -1
        return neg_rvs

    def check_sum(self):
        sum_checker = np.sum(self.weights) - 1
        
        return sum_checker

    def finalPortfolio(self):
        opt_results = minimize(self.negative_sharpe(), [0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555]
        , method = 'SLSQP'
        , bounds=[(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)]
        , constraints=({'type':'eq','fun': self.check_sum()}))
        weights = opt_results.x
        portfolio_df = pd.DataFrame(self.tickers)
        portfolio_df['weights'] = weights
        portfolio_df = portfolio_df.rename(columns={0:'ticker'})
        print(portfolio_df)
        return portfolio_df
        

if __name__ == "__main__":
    gc = gs.service_account(filename='dva-api-384400-c063824c0207.json')
    sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1nS8WRaVB5S2aaxMiFpVHxSrEE12lx6TGs_WyAkcUSzU/edit?usp=sharing')
    ws = sh.worksheet('Form Responses 1')
    df = pd.DataFrame(ws.get_all_records())
    df = df[['Timestamp', 'Risk Assessment Score']].rename(columns={'Timestamp':'time', 'Risk Assessment Score':'score'})
    df = df.sort_values(by='time', ascending=False).head(1)
    pb = PortfolioBalancer(df)
    print(pb.finalPortfolio())

In [192]:
gc = gs.service_account(filename='/Users/suhailp/Documents/gt_projects/cse6242/DVA/dva-api-384400-c063824c0207.json')
gc

sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1nS8WRaVB5S2aaxMiFpVHxSrEE12lx6TGs_WyAkcUSzU/edit?usp=sharing')
ws = sh.worksheet('Form Responses 1')

df = pd.DataFrame(ws.get_all_records())

df = df[['Timestamp', 'Risk Assessment Score']].rename(columns={'Timestamp':'time', 'Risk Assessment Score':'score'})
df = df.sort_values(by='time', ascending=False).head(1)
df

,time,score
2,4/20/2023 20:08:25,42


In [259]:
if df['score'].iloc[0] <= 12:
    tickers = ['aapl','adbe','amd','csco', 'fisv', 'ibm', 'intc', 'lrcx', 'msft', 'mu', 'orcl', 'qcom', 'txn', 'nvda', 'fis', 'crm', 'avgo', 'now']
    
elif (df['score'].iloc[0] > 12) & (df['score'].iloc[0] <= 20):
    tickers = ['csgp','mdu','amd','csco', 'lly', 'ibm', 'intc', 'wba', 'isrg', 'pep', 'orcl', 'qcom', 'aciw', 'nvda', 'qlys', 'crm', 'cost', 'now']

elif (df['score'].iloc[0] > 20) & (df['score'].iloc[0] <= 28):
    tickers = ['aapl','adbe','amd','csco', 'fisv', 'ibm', 'intc', 'lrcx', 'msft', 'mu', 'orcl', 'nbix', 'txn', 'nvda', 'fis', 'crm', 'avgo', 'saic']

elif (df['score'].iloc[0] > 29) & (df['score'].iloc[0] <= 34):
    tickers = ['dci','adbe','mrcy','wtrg', 'fisv', 'ibm', 'axs', 'lrcx', 'msft', 'bros', 'orcl', 'pypl', 'txn', 'amrc', 'fis', 'crm', 'chgg', 'now']
    
elif (df['score'].iloc[0] > 35):
    tickers = ['meta','adbe','amd','csco', 'fisv', 'ibm', 'ale', 'lrcx', 'jazz', 'mu', 'orcl', 'qcom', 'txn', 'nvda', 'fis', 'crm', 'eqix', 'vrsk']

stock_data = yf.download(tickers,start='2017-01-1', end='2018-12-1')
stock_data = stock_data.dropna()
stocks = stock_data['Adj Close']
stocks

[*********************100%***********************]  18 of 18 completed


,ADBE,ALE,AMD,CRM,CSCO,EQIX,FIS,FISV,IBM,JAZZ,LRCX,META,MU,NVDA,ORCL,QCOM,TXN,VRSK
Date,,,,,,,,,,,,,,,,,,
2017-01-03,103.480003,51.007633,11.430000,70.540001,24.954922,319.760590,70.438599,53.669998,119.605850,111.459999,97.231216,116.860001,22.276924,25.142954,34.786842,54.375095,62.358192,79.059433
2017-01-04,104.139999,51.440308,11.430000,72.800003,24.806576,326.169678,71.591522,54.325001,121.086655,115.830002,97.285606,118.690002,22.089224,25.729559,34.958290,54.433296,62.281914,79.681641
2017-01-05,105.910004,51.111801,11.240000,72.790001,24.864267,329.592621,71.417679,54.305000,120.686073,115.750000,97.068016,120.669998,21.842251,25.076406,34.868053,54.499813,61.798836,79.389984
2017-01-06,108.300003,50.991615,11.320000,73.800003,24.913715,330.252167,72.186302,54.915001,121.279831,115.160004,97.539406,123.410004,21.773098,25.411608,34.696606,54.483177,62.841236,80.089951
2017-01-09,108.570000,49.933960,11.490000,73.959999,24.872511,332.658905,71.408546,54.049999,119.934898,116.580002,98.681610,124.900002,22.069466,26.441874,35.219986,54.582966,63.002254,79.458023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-11-26,231.960007,67.065033,20.080000,126.410004,39.739693,355.094971,97.668472,77.410004,92.868179,150.960007,136.701691,136.380005,36.127140,37.937729,44.783421,49.522591,85.919098,117.587761
2018-11-27,232.250000,67.379784,21.049999,127.540001,40.219322,355.039154,97.415855,77.169998,93.233253,149.929993,136.729584,135.000000,36.551937,38.106285,44.022503,49.380409,85.829956,118.307213
2018-11-28,249.210007,68.221909,21.340000,140.639999,41.239635,356.851074,100.999252,79.400002,95.540184,146.429993,143.257370,136.759995,38.241226,39.677830,44.718460,50.340115,87.595078,120.358543


In [240]:
# Set seed (optional)
np.random.seed(100)

log_ret = np.log(stocks/stocks.shift(1))
log_ret.head()

w_random = np.random.random(18)
weights = np.array(w_random)
print(weights/np.sum(weights))
print('\n\n')

weight_log = log_ret.mean() * weights
expected_returns = np.sum(weight_log) *252
print(expected_returns)
print('\n')

transformed_weight = weights.T
day_scale = 252
expected_volatility = np.sqrt(np.dot(transformed_weight, np.dot(log_ret.cov() * day_scale, weights)))
print(expected_volatility)
print('\n')

sharpe_ratio = exp_ret/exp_vol
print(sharpe_ratio)

[0.06790921 0.03478777 0.05305188 0.10557152 0.00058971 0.01519247
 0.08382339 0.10320666 0.0170842  0.07186931 0.11138834 0.02614395
 0.02316043 0.01354384 0.02745555 0.12229843 0.10143589 0.02148744]



1.4010219709196143


1.4155129886346502


1.1381733563327558


In [248]:
num_of_stocks = len(stocks.columns)
all_weights = np.zeros((15000, num_of_stocks))
ret_arr = np.zeros(num_ports)
vol_arr = np.zeros(num_ports)
sharpe_arr = np.zeros(num_ports)
day_len = 252
w_transform = weights.T
retention_mean = log_ret.mean()

for indicator in range(num_ports):
    
    weights = np.array(np.random.random(18))
    all_weights[indicator,:] = weights / np.sum(weights)
    ret_arr[indicator] = np.sum((retention_mean * weights) * day_len)
    vol_arr[indicator] = np.sqrt(np.dot(w_transform, np.dot(log_ret.cov() * day_len, weights)))
    sharpe_arr[indicator] = ret_arr[indicator]/vol_arr[indicator]

In [257]:
def get_rvs(weights):
    weights = np.array(weights)
    ret = np.sum(log_ret.mean() * weights) * 252
    vol = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))
    sr = ret/vol
    return np.array([ret,vol,sr])

def negative_sharpe(weights):
    rvs_return = get_rvs(weights)[2]
    neg_rvs = rvs_return * -1
    return neg_rvs

def check_sum(weights):
    sum_checker = np.sum(weights) - 1
    
    return sum_checker

opt_results = minimize(negative_sharpe, [0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555,0.0555], method = 'SLSQP', bounds=[(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)], constraints=({'type':'eq','fun': check_sum}))
opt_results

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -1.823114086079825
       x: [ 3.534e-01  0.000e+00 ...  1.163e-16  0.000e+00]
     nit: 7
     jac: [ 1.338e-04  3.843e-01 ...  1.250e+00  3.573e-01]
    nfev: 134
    njev: 7

In [258]:
weights = opt_results.x

portfolio_df = pd.DataFrame(tickers)

portfolio_df['weights'] = weights
portfolio_df = portfolio_df.rename(columns={0:'ticker'})
portfolio_df

,ticker,weights
0,meta,3.534151e-01
1,adbe,0.000000e+00
2,amd,0.000000e+00
3,csco,0.000000e+00
4,fisv,1.770727e-01
5,ibm,0.000000e+00
6,intc,2.607481e-01
7,lrcx,6.712431e-16
8,msft,0.000000e+00
9,mu,0.000000e+00


In [233]:
gc = pygsheets.authorize(service_file='/Users/suhailp/Documents/gt_projects/cse6242/DVA/dva-api-384400-c063824c0207.json')

#open the google spreadsheet 
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1vJyfYKYi0q9KeQyyMRiBA62-jmRrCnw4d4o7UikCbK8/edit?usp=sharing')

#select the first sheet 
wks = sh[0]

#update the first sheet with df, starting at cell B2. 
wks.set_dataframe(portfolio_df, 'A1')

In [ ]:
tables = jupytab.Tables()
tables['dynamic'] = jupytab.DataFrameTable('new balanced portfolio', refresh_method=pd.DataFrame())